## 1. Import

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import random
import os



In [2]:
import importlib
import connect_four
import random_bot
import logic_bot
importlib.reload(connect_four)
importlib.reload(random_bot)
importlib.reload(logic_bot)
from connect_four import ConnectFour
from random_bot import RandomBot
from logic_bot import LogicBot
from logic_bot import MinMaxBot

## 2. NN Bot Class

In [3]:
import nn_bot
importlib.reload(nn_bot)
from nn_bot import NeuralNetworkBot


## Functions

In [4]:
def initialize_population(population_size, piece):
    return [NeuralNetworkBot(piece) for _ in range(population_size)]

In [6]:
def evaluate_fitness(bot, opponents, games=10):
    fitness = 0
    opponent = random.choice(opponents)
    for i in range(games):
        game = ConnectFour()
        while game.state == 'UNFINISHED':
            col = bot.choose_column(game)
            game.insert(col, bot.piece)
            if game.state != 'UNFINISHED':
                # print(game)
                fitness += 1 if game.state == bot.piece + '_WON' else 0
                fitness += 0 if game.state == 'DRAW' else 0
                break

            col = opponent.choose_column(game)
            game.insert(col, opponent.piece)
            if game.state != 'UNFINISHED':
                # print(game)
                fitness += -1 if game.state == opponent.piece + '_WON' else 0
                fitness += 0 if game.state == 'DRAW' else 0
                break
    return fitness

In [7]:
def select_top_performers(population, fitnesses, num_top):
    sorted_indices = np.argsort(fitnesses)[::-1]
    return [population[i] for i in sorted_indices[:num_top]]

In [8]:
def generate_new_population(top_performers, population_size, mutation_rate=0.01):
    new_population = []
    while len(new_population) < population_size:
        if len(top_performers) > 1:
            parent1, parent2 = random.sample(top_performers, 2)
        else:
            parent1 = parent2 = top_performers[0]
        child = parent1.crossover(parent2)
        child.mutate(mutation_rate)
        new_population.append(child)
    return new_population

In [9]:
# def train_evolutionary(population_size=50, generations=100, num_top=10, mutation_rate=0.01, games_per_fitness=10):
#     population = initialize_population(population_size, 'X')
#     opponent = RandomBot('O')

#     for generation in range(generations):
#         fitnesses = [evaluate_fitness(bot, opponent, games_per_fitness) for bot in population]
#         top_performers = select_top_performers(population, fitnesses, num_top)
#         population = generate_new_population(top_performers, population_size, mutation_rate)
#         print(f'Generation {generation + 1}, Best Fitness: {max(fitnesses)}')

#     return population
def train_evolutionary(population_size=50, generations=100, games_per_fitness=10, num_top=10, mutation_rate=0.01, opponent=None, prev_population=None):
    # Initialize population with random bots
    populations = []
    population = [NeuralNetworkBot('X') for _ in range(population_size)] if not prev_population else prev_population
    if not opponent:
        opponent = RandomBot('O')
    # previous_top_performers = None
    populations.append(population)
    best_performer = []

    for generation in range(generations):
        fitnesses = [evaluate_fitness(bot, [opponent], games_per_fitness) for bot in population]

        # if previous_top_performers:
        #     fitnesses_prev = [evaluate_fitness(bot, previous_top_performers, games_per_fitness // 2) for bot in population]
        #     fitnesses = [f1 + f2 for f1, f2 in zip(fitnesses, fitnesses_prev)]

        top_performers = select_top_performers(population, fitnesses, num_top)
        # previous_top_performers = [NeuralNetworkBot('O', bot.model) for bot in top_performers]

        population = generate_new_population(top_performers, population_size, mutation_rate)
        populations.append(population)
        best_performer.append(fitnesses.index(max(fitnesses)))
        print(f"Generation {generation}: Best fitness = {max(fitnesses)} Average fitness = {np.mean(fitnesses)}")
        sorted_fitness = sorted(fitnesses)
        # print(f"Fitness: {sorted_fitness[-5:]}")
        print(f"Fitness: {fitnesses}")

    return populations, best_performer

In [9]:
# parallel version
from concurrent.futures import ProcessPoolExecutor

def evaluate_fitness_parallel(args):
    return evaluate_fitness(*args)

def train_parallel_evolutionary(population_size=20, generations=100, games_per_fitness=20, num_top=5, mutation_rate=0.05, opponent=None, prev_population=None):
    population = [NeuralNetworkBot('X') for _ in range(population_size)] if not prev_population else prev_population
    if not opponent:
        opponent = RandomBot('O')
    previous_top_performers = None
    best_fitness_over_generations = []

    with ProcessPoolExecutor() as executor:
        for generation in range(generations):
            fitness_args = [(bot, opponent, games_per_fitness) for bot in population]
            fitnesses = list(executor.map(evaluate_fitness_parallel, fitness_args))

            if previous_top_performers:
                fitness_args_prev = [(bot, previous_top_performers, games_per_fitness // 2) for bot in population]
                fitnesses_prev = list(executor.map(evaluate_fitness_parallel, fitness_args_prev))
                fitnesses = [f1 + f2 for f1, f2 in zip(fitnesses, fitnesses_prev)]

            top_performers = select_top_performers(population, fitnesses, num_top)
            previous_top_performers = top_performers
            population = generate_new_population(top_performers, population_size, mutation_rate)
            best_fitness = max(fitnesses)
            best_fitness_over_generations.append(best_fitness)
            print(f"Generation {generation}: Best fitness = {best_fitness}")

    return population, best_fitness_over_generations

In [10]:
import os
from datetime import datetime
import tensorflow as tf

def save_models(trained_populations):
    now = datetime.now()
    dir_name = os.path.join('models', now.strftime("%Y-%m-%d-%H-%M"))
    os.makedirs(dir_name, exist_ok=True)
    
    for generation, population in enumerate(trained_populations):
        if generation > 0:
            for ind, bot in enumerate(population):
                model_path = os.path.join(dir_name, f'gen{generation}_model{ind}.keras')
                tf.keras.models.save_model(bot.model, model_path, include_optimizer=True)

# Example usage:
# save_models(trained_populations)

def load_model(model_path):
    tf.keras.backend.clear_session()  # Clear the current session to avoid conflicts
    model = tf.keras.models.load_model(model_path)
    # model.summary()  # Print the model summary to verify the architecture
    return model

# Example usage:
# model_path = 'path/to/saved/model/gen1_model0.keras'
# model = load_model(model_path)

In [29]:
base_path = r"C:\Users\storm\Documents\GitHub\Connect4\Connect-4-Game\models\2024-06-13-13-42"
prev_population = []
for i in range(50):
    model_path = os.path.join(base_path, f"gen30_model{i}.keras")
    model = load_model(model_path)
    model.compile(optimizer=Adam(), loss='mse')
    prev_population.append(NeuralNetworkBot('X', model))

c:\Users\storm\Documents\GitHub\Connect4\Connect-4-Game\.venv\lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 34 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [22]:
# prev_population = None
if prev_population:
    print("Loaded previous population")
    trained_populations, best_performers = train_evolutionary(population_size=50, generations=100, games_per_fitness=5, num_top=20, mutation_rate=0.01, opponent=LogicBot('O'), prev_population=prev_population)
else:
    print("No previous population")
    # trained_populations, best_performers = train_evolutionary(population_size=10, generations=10, games_per_fitness=5, num_top=5, mutation_rate=0.01, opponent=MinMaxBot('O', 1))
    trained_populations, best_performers = train_evolutionary(population_size=50, generations=30, games_per_fitness=5, num_top=25, mutation_rate=0.01, opponent=RandomBot('O'))
save_models(trained_populations)
prev_population = trained_populations[-1]

Generation 0: Best fitness = 5 Average fitness = 2.72
Fitness: [3, 3, 1, 3, 1, 3, 1, 3, 3, 1, 1, 5, 3, 1, 3, 5, 3, 5, 1, 3, 1, 5, 1, 3, -1, 3, 1, 1, 3, -1, 3, 3, 1, 3, 3, 3, 3, 1, 3, 5, 1, 5, 5, 5, 1, 5, 5, 5, 5, 3]
Generation 1: Best fitness = 5 Average fitness = 2.72
Fitness: [3, 5, 3, 5, 3, 5, 1, 1, 3, 1, 5, 5, -3, 3, -1, -1, 1, 1, 5, 5, 5, 3, 5, 1, 1, 3, 5, 5, -1, 5, 1, 5, 1, 5, 3, 3, 1, 1, 5, 1, 5, -1, 1, 3, 3, 5, 3, 3, 5, 1]
Generation 2: Best fitness = 5 Average fitness = 3.16
Fitness: [3, 3, 1, 5, 1, 5, 3, 3, 3, 3, 3, 5, 5, 3, 5, 3, 1, 3, 5, 5, 5, 5, 3, 3, 3, 3, 3, -1, 1, 3, 1, 1, 5, 5, 3, 1, 1, 5, 5, 1, 3, 1, 5, 3, 5, 3, 5, 3, 1, 5]
Generation 3: Best fitness = 5 Average fitness = 3.24
Fitness: [3, -1, 3, 3, 3, -1, 5, -1, 5, 5, 3, 5, 3, 3, 3, 1, 3, 3, 3, 3, 5, 3, 3, 5, 5, 3, 1, 3, 3, 3, 1, 1, 5, 3, 3, 1, 3, 5, 3, 5, 5, 5, 3, 5, 5, 5, 3, 3, 5, 5]
Generation 4: Best fitness = 5 Average fitness = 3.2
Fitness: [5, 5, 5, 1, -1, 3, 5, 3, -1, 5, 5, 3, 3, 5, 3, 3, 3, 3, 5, -1, 5, 1, 3

In [23]:
trained_populations = train_parallel_evolutionary(population_size=50, generations=5, games_per_fitness=20, num_top=10, mutation_rate=0.01, opponent=LogicBot('O'))

NameError: name 'train_parallel_evolutionary' is not defined

In [26]:
import os
from datetime import datetime

now = datetime.now()
dir_name = os.path.join('models', now.strftime("%Y-%m-%d-%H-%M"))

os.makedirs(dir_name, exist_ok=True)
for generation, population in enumerate(trained_populations):
    if generation > 0:
        for ind, bot in enumerate(population):
            model_path = os.path.join(dir_name, f'gen{generation}_model{ind}.keras')
            # bot.model.save(model_path)
            tf.keras.models.save_model(bot.model, model_path, include_optimizer=True)




In [11]:
trained_populations, best_performers = train_evolutionary(population_size=50, generations=30, games_per_fitness=5, num_top=25, mutation_rate=0.01, opponent=RandomBot('O'))
save_models(trained_populations)
prev_population = trained_populations[-1]
trained_populations, best_performers = train_evolutionary(population_size=50, generations=100, games_per_fitness=5, num_top=25, mutation_rate=0.01, opponent=LogicBot('O'), prev_population=prev_population)

Generation 0: Best fitness = 5 Average fitness = 2.92
Fitness: [3, 3, 3, 3, 5, 5, 3, -3, 1, 3, 5, 3, 1, 3, 5, 3, 1, 3, 3, 3, 1, 3, 3, 1, 5, 5, 1, 3, 5, 3, 5, 1, 3, 5, 1, 3, 3, 3, 5, 3, 3, -1, 5, 3, 5, -1, 3, 3, 5, 3]
Generation 1: Best fitness = 5 Average fitness = 2.56
Fitness: [1, 1, 5, 1, 3, 1, 3, 3, 5, 3, 5, 5, -1, -1, 3, 3, 5, 5, 3, 1, 5, 5, 5, 1, 3, 5, 1, 3, 3, 3, 1, 3, 5, 1, 1, 1, 5, -3, 1, 3, 5, 1, 3, 1, 5, 1, 1, 1, 3, 1]
Generation 2: Best fitness = 5 Average fitness = 3.32
Fitness: [-1, 3, 3, 3, 3, 1, 3, 5, 5, 5, -3, -1, 5, 5, 5, 3, 3, -3, 5, 3, 5, 5, 3, 1, 3, 5, 1, 3, 3, 3, 5, 5, 3, 5, 5, 3, 5, 3, 5, 1, 5, 5, 5, 3, 5, 5, 5, 3, 3, 3]
Generation 3: Best fitness = 5 Average fitness = 2.48
Fitness: [3, 1, 3, 5, 3, 3, 1, 3, 5, 1, -1, 3, 3, 5, 1, 3, -3, 3, -1, 1, 5, 5, 3, -1, 5, 3, 3, 1, 3, 1, 5, 5, 5, 1, 5, 5, 1, 1, 5, 1, -3, 3, 5, 3, -1, 3, -1, 3, 3, 5]
Generation 4: Best fitness = 5 Average fitness = 3.16
Fitness: [3, 5, 3, 3, 3, 5, 5, 3, 3, 5, 5, 5, 3, 5, 3, 5, -1, 5, 1, 5, -1

In [ ]:
prev_population = trained_populations[-1] if trained_populations else None
trained_populations, best_performers = train_evolutionary(population_size=50, generations=100, games_per_fitness=5, num_top=25, mutation_rate=0.01, opponent=LogicBot('O'), prev_population=prev_population)

In [12]:
save_models(trained_populations)